In [1]:
# TODO: will change to use "pip install -e ." for developing the package
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

In [2]:
import time
import re

import pandas as pd
from collections import OrderedDict

import json
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
from deck_crawler.parse_deck import parse_events_from_official
from deck_crawler.parse_deck import reassign_category
from deck_crawler.utils.translator import translate_deck

In [4]:
# flags and parameters
RUN_PARSE = False
DEBUG_PARSE = False

NUM_RESULT_PAGE = 3
NUM_EVENT_PAGE = 1000
NUM_DECK_PAGE = 2

START_DATE = "2023年02月04日"  # include
END_DATE = "2023年03月04日"  # include

TARGET_CATEGORY = "密勒頓ex"

"""
放逐_騎拉帝納VSTAR

LTB
LTB_勾魂眼_噴火龍
LTB_蓋歐卡
LTB_洗翠黏美龍VSTAR

阿爾宙斯VSTAR
阿爾_騎拉帝納VSTAR
阿爾_白馬VMAX
阿爾_鋁鋼龍VMAX
純阿爾VSTAR

夢幻VMAX

密勒頓ex

洛奇亞VSTAR
一擊_洛奇亞

沙奈朵ex
沙奈朵_蒼響
沙奈朵_超夢V-UNION

起源帕路奇亞VSTAR

連擊武道熊師VMAX

達克萊伊VSTAR

洗翠索羅亞克VSTAR

--

飄香豚ex

雷吉鐸拉戈VSTAR

烈空坐VMAX

火焰雞VMAX



起源帝牙盧卡VSTAR	1

索羅風速狗	1

洛托姆VSTAR	1

月石	1

化石翼龍VSTAR	1
"""

'\n放逐_騎拉帝納VSTAR\n\nLTB\nLTB_勾魂眼_噴火龍\nLTB_蓋歐卡\nLTB_洗翠黏美龍VSTAR\n\n阿爾宙斯VSTAR\n阿爾_騎拉帝納VSTAR\n阿爾_白馬VMAX\n阿爾_鋁鋼龍VMAX\n純阿爾VSTAR\n\n夢幻VMAX\n\n密勒頓ex\n\n洛奇亞VSTAR\n一擊_洛奇亞\n\n沙奈朵ex\n沙奈朵_蒼響\n沙奈朵_超夢V-UNION\n\n起源帕路奇亞VSTAR\n\n連擊武道熊師VMAX\n\n達克萊伊VSTAR\n\n洗翠索羅亞克VSTAR\n\n--\n\n飄香豚ex\n\n雷吉鐸拉戈VSTAR\n\n烈空坐VMAX\n\n火焰雞VMAX\n\n\n\n起源帝牙盧卡VSTAR\t1\n\n索羅風速狗\t1\n\n洛托姆VSTAR\t1\n\n月石\t1\n\n化石翼龍VSTAR\t1\n'

In [5]:
# create some folders
EXCEL_FOLDER = "excel"
DB_FOLDER = "deck_db"
folders = [EXCEL_FOLDER, DB_FOLDER]

for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

if not os.path.exists(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}"):
    os.makedirs(f"{EXCEL_FOLDER}/{TARGET_CATEGORY}")

In [6]:
# loading db
decks = {}
store_file_name = "deck_db/test.json"
if not DEBUG_PARSE:
    store_file_name = "deck_db/store.json"
    if os.path.exists(store_file_name):
        with open(store_file_name, "r") as f:
            decks = json.load(f)

decks = reassign_category(decks)
print("categories in previous result:")
print(decks.keys())

store_code_list = []
for category in decks.keys():
    for d in decks[category]:
        store_code_list.append(d["deck_code"])
print("\n")
print("number of decks in the previous result:")
print(len(store_code_list))

categories in previous result:
dict_keys(['夢幻VMAX', '交錯切換機', '化石翼龍VSTAR', 'LTB', 'LTB_勾魂眼_噴火龍', '雷吉艾勒奇VMAX', '密勒頓ex', 'Other_Lost', 'LTB_洗翠黏美龍VSTAR', '達克萊伊VSTAR', '起源帕路奇亞VSTAR', '白馬蕾冠王VMAX', '放逐_騎拉帝納VSTAR', '超夢 V-UNION', '阿爾宙斯VSTAR', '光輝無極汰那', '飛翔皮卡丘VMAX', '酋雷姆VMAX', '起源帝牙盧卡VSTAR', 'LTB_空之封印石_水超鋼雷', 'LTB_空之封印石_水超闘雷', 'LTB_蓋歐卡', 'LTB_空之封印石_other', 'LTB_空之封印石_超鋼雷', '阿羅拉六尾VSTAR', '阿爾_白馬VMAX', 'LTB_空之封印石_水超', '阿爾_飛天皮VMAX', '連擊武道熊師VMAX', 'LTB_空之封印石_超闘', 'LTB_空之封印石_水超鋼闘', 'LTB_空之封印石_水超闘', 'LTB_空之封印石_悪水超雷', 'LTB_快龍V', 'LTB_空之封印石_悪水超鋼', 'LTB_空之封印石_水超鋼闘雷', 'LTB_空之封印石_草超鋼雷', 'LTB_空之封印石_水超雷', 'LTB_空之封印石_超闘雷', 'LTB_空之封印石_草超鋼闘雷', 'LTB_空之封印石_超鋼闘雷', 'LTB_空之封印石_水草超闘雷', 'LTB_空之封印石_水超鋼', 'LTB_空之封印石_草超闘雷', 'LTB_空之封印石_水草超鋼雷', 'LTB_空之封印石_水草超鋼闘雷', 'LTB_空之封印石_悪水超闘', 'LTB_空之封印石_悪水超', '沙奈朵ex', 'LTB_空之封印石_超', '沙奈朵_蒼響', 'LTB_空之封印石_悪水超鋼闘', '洛托姆VSTAR', 'カラマネロVMAX', 'LTB_古月鳥', 'LTB_噴火龍', '阿爾_騎拉帝納VSTAR', '雷吉鐸拉戈VSTAR', '洛奇亞VSTAR', '沙奈朵_超夢V-UNION', '黑馬蕾冠王VMAX', 'Other_騎拉帝納VSTAR', '純阿爾VSTAR', '阿爾_鋁鋼龍VMAX', '烈空坐VMAX', '

In [7]:
# parse events
t1 = time.time()
if RUN_PARSE or DEBUG_PARSE:
    parse_events_from_official(
        decks,
        store_code_list,
        result_page_limit=NUM_RESULT_PAGE,
        event_page_limit=NUM_EVENT_PAGE,
        deck_page_limit=NUM_DECK_PAGE,
    )
t2 = time.time()

print()
print(f"{t2 - t1} seconds")


7.82012939453125e-05 seconds


In [8]:
# save to json
with open(store_file_name, "w") as f:
    json.dump(decks, f, ensure_ascii=False, indent=4)

In [9]:
# # show all categories we have
# total = 0
# for k in decks.keys():
#     print(f"[{k}]:\n{len(decks[k])}\n")
#     total += len(decks[k])

# print(total)

In [10]:
# show categories we have in the period
real_start_date, real_end_date = "3000年01月01日", "1000年01月01日"
number_target_deck = 0
deck_cnt_list = []
for k in decks.keys():
    deck_cnt = 0
    for deck in decks[k]:
        date_exclude_jp_str = deck["date"][:-3]
        if date_exclude_jp_str >= START_DATE and date_exclude_jp_str <= END_DATE:
            deck_cnt += 1

            if date_exclude_jp_str < real_start_date:
                real_start_date = date_exclude_jp_str

            if date_exclude_jp_str > real_end_date:
                real_end_date = date_exclude_jp_str

    # print(f"{k}\t{deck_cnt}\n")
    deck_cnt_list.append((deck_cnt, k))
    if k == TARGET_CATEGORY:
        number_target_deck = deck_cnt

In [11]:
deck_cnt_list = sorted(deck_cnt_list, reverse=True)
for num_deck, deck_name in deck_cnt_list:
    if num_deck > 0:
        print(f"{deck_name}\t{num_deck}\n")

放逐_騎拉帝納VSTAR	445

LTB	346

密勒頓ex	283

阿爾宙斯VSTAR	282

雷吉艾勒奇VMAX	269

交錯切換機	269

夢幻VMAX	238

洛奇亞VSTAR	236

一擊_洛奇亞	221

Other_騎拉帝納VSTAR	135

飛翔皮卡丘VMAX	131

LTB_勾魂眼_噴火龍	131

沙奈朵ex	109

阿爾_騎拉帝納VSTAR	87

連擊武道熊師VMAX	70

沙奈朵_蒼響	70

LTB_蓋歐卡	61

阿爾_飛天皮VMAX	57

白馬蕾冠王VMAX	53

超夢 V-UNION	50

起源帕路奇亞VSTAR	50

沙奈朵_超夢V-UNION	46

洗翠索羅亞克VSTAR	34

阿爾_鋁鋼龍VMAX	31

阿爾_白馬VMAX	25

黑馬蕾冠王VMAX	18

LTB_洗翠黏美龍VSTAR	18

達克萊伊VSTAR	17

純阿爾VSTAR	15

others	15

Other_Lost	14

LTB_空之封印石_水超闘雷	14

LTB_空之封印石_other	10

LTB_快龍V	10

起源帝牙盧卡VSTAR	9

火焰雞VMAX	9

光輝無極汰那	9

LTB_空之封印石_水超鋼闘	9

酋雷姆VMAX	7

洛托姆VSTAR	7

飄香豚ex	6

烈空坐VMAX	5

雷吉鐸拉戈VSTAR	4

藏飽栗鼠VMAX	4

LTB_空之封印石_水超闘	4

阿羅拉六尾VSTAR	3

LTB_空之封印石_水超雷	3

LTB_空之封印石_水超鋼雷	3

LTB_空之封印石_水超	3

索羅風速狗	2

月石	2

幻影索羅亞克	2

化石翼龍VSTAR	2

LTB_空之封印石_水超鋼闘雷	2

LTB_空之封印石_悪水超鋼闘	2

柱神	1

ヒスイ ダイケンキVSTAR	1

ハピナスV	1

LTB_空之封印石_超闘	1

LTB_空之封印石_悪水超雷	1

LTB_空之封印石_悪水超闘	1

LTB_空之封印石_悪水超鋼	1

LTB_空之封印石_悪水超	1

LTB_噴火龍	1



In [12]:
# number of decks for the target category
print(number_target_deck)
if number_target_deck <= 0:
    raise Exception(f"No decks for {TARGET_CATEGORY} during {START_DATE} to {END_DATE}")

283


In [13]:
# Analysis
df_list = []
common_cols = ["date", "prefecture", "num_players", "rank"]
int_cols = ["num_players", "rank"]

for card_type in ["pokemons", "tools", "supporters", "stadiums", "energies"]:
    # df init
    df = pd.DataFrame()
    for _, deck in enumerate(decks[TARGET_CATEGORY]):
        deck = translate_deck(deck)

        deck_code = deck["deck_code"]  # row id
        date_exclude_jp_str = deck["date"][:-3]
        if date_exclude_jp_str < real_start_date or date_exclude_jp_str > real_end_date:
            continue
        
        pokecard = OrderedDict()
        for col in common_cols:
            pokecard[col] = deck.get(col, "")
        pokecard.update(deck[card_type])

        if _ == 0:
            df = pd.DataFrame(pokecard, index=[deck_code])
        else:
            df = pd.concat([df, pd.DataFrame(pokecard, index=[deck_code])])
    df = df.fillna(0)

    # sort rows by date
    df = df.sort_values(by=["date"], ascending=False)

    # select cols for analysis
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)

    # calculate
    num_decks = df.shape[0]
    num_used = df[col_list].sum(axis="rows", numeric_only=True)
    num_picked = df[col_list].astype(bool).sum(axis="rows")
    avg_num_used = num_used / num_picked
    pick_rate = num_picked / num_decks

    # insert rows in df
    df.loc["avg_num_used"] = {}
    df.loc["pick_rate"] = {}
    for col in col_list:
        df.loc["avg_num_used", col] = avg_num_used[col]
        df.loc["pick_rate", col] = pick_rate[col]

    # reorder index in df, move 'avg_num_used' and 'pick_rate' to top
    num_rows = df.shape[0]
    target_rows = [num_rows - 1, num_rows - 2]
    idx = target_rows + [i for i in range(len(df)) if i not in target_rows]
    df = df.iloc[idx]

    # sort cols by pick rate
    df = df.sort_values("pick_rate", axis=1, ascending=False)
    col_list = list(df)
    for c in common_cols:
        col_list.remove(c)
    df = df[common_cols + col_list]

    # format data type
    for col in int_cols:
        df[col] = df[col].astype("Int64")

    # store
    df_list.append(df)

In [14]:
# excel writer
writer = pd.ExcelWriter(
    f"{EXCEL_FOLDER}/{TARGET_CATEGORY}/{TARGET_CATEGORY}-{real_start_date}-{real_end_date}.xlsx",
    engine="xlsxwriter",
    mode="w",
)
workbook = writer.book

for sheet_id, sheet_name in enumerate(
    ["pokemons", "tools", "supporters", "stadiums", "energies"]
):
    df_list[sheet_id].to_excel(writer, sheet_name=sheet_name, float_format="%.3f")

    # Make deck_link become a hyperlink
    # Get the xlsxwriter workbook and worksheet objects
    worksheet = writer.sheets[sheet_name]
    for i, deck_id in enumerate(df_list[sheet_id].index):
        if deck_id in ["avg_num_used", "pick_rate"]:
            continue

        # Calculate the row number
        row = i + 2

        # Write the hyperlink to the cell
        worksheet.write_url(
            f"A{row}",
            f"https://www.pokemon-card.com/deck/confirm.html/deckID/{deck_id}",
            string=f"{deck_id}",
        )

    # formatting
    header_format = workbook.add_format(
        {"bold": True, "text_wrap": True, "valign": "top"}
    )
    for col_num, value in enumerate(df_list[sheet_id].columns.values):
        worksheet.write(0, col_num + 1, value, header_format)
    worksheet.autofit()

workbook.close()
writer.save()

/Users/blake.pan/anaconda3/envs/sandbox_crawler/lib/python3.7/site-packages/xlsxwriter/workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [15]:
df_list[0]

,date,prefecture,num_players,rank,密勒頓ex\nSV1V 037/078,雷吉艾勒奇V\nS12 033/098,雷吉艾勒奇VMAX\nS12 034/098,雷公V\nS12a 038/172,龍王蠍V\nS12a 085/172,鑰圈兒\nSV1V 045/078,光輝甲賀忍蛙\nS12a 033/172,茸茸羊\nS12a 037/172,咩利羊\nS12a 036/172,飛翔皮卡丘V\nS8a 023/028,飛翔皮卡丘VMAX\nS8a 024/028,倫琴貓V\nS10D 021/067,雷丘V\nS9 034/100,霓虹魚V\nSPD 002/020,自爆磁怪V\nS10a 016/071,自爆磁怪VSTAR\nS10a 017/071,帕奇利茲\nSV1V 029/078,電龍ex\nSVAL 007/021,咩利羊\nSVAL 005/021,瑪納霏\nS12a 029/172,咩利羊\nS8b 051/184,雷吉艾勒奇\nS7R 033/067,光輝甜冷美后\nS11a 009/068,雷電雲\nS5a 022/070,閃電鳥\nS10b 029/071,夢幻\nS12a 052/172,大牙狸\nS12a 121/172,大尾狸\nS9 082/100,電飛鼠\nS6K 023/070,洛托姆V\nS11 037/100,洛托姆VSTAR\nSP6 002/006,捷拉奧拉\nS12 032/098,伽勒爾 急凍鳥V\nS8b 230/184,花療環環\nS11 049/100,古月鳥\nS11 033/100,音箱蟀V\nSI 014/414,伽勒爾 閃電鳥V\nS8b 084/184,密勒頓ex\nSV1V 102/078,茸茸羊\nSK 006/030,光輝噴火龍\nS10b 011/071,摔角鷹人\nSV1S 045/078
pick_rate,NaN,NaN,<NA>,<NA>,0.996466,0.950530,0.950530,0.749117,0.639576,0.462898,0.349823,0.328622,0.265018,0.250883,0.250883,0.247350,0.098940,0.091873,0.060071,0.056537,0.042403,0.042403,0.042403,0.038869,0.031802,0.028269,0.021201,0.017668,0.017668,0.014134,0.010601,0.010601,0.007067,0.003534,0.003534,0.003534,0.003534,0.003534,0.003534,0.003534,0.003534,0.003534,0.003534,0.003534,0.003534
avg_num_used,NaN,NaN,<NA>,<NA>,3.028369,3.553903,3.434944,1.089623,1.011050,2.320611,1.000000,1.537634,1.453333,1.014085,1.014085,1.528571,1.285714,1.000000,1.294118,1.250000,1.000000,1.000000,1.666667,1.000000,2.000000,1.250000,1.000000,1.000000,1.200000,1.250000,1.333333,1.333333,1.000000,4.000000,3.000000,1.000000,1.000000,4.000000,2.000000,1.000000,1.000000,3.000000,3.000000,1.000000,1.000000
FFff5f-yNt4hB-kwF1Ff,2023年03月04日(土),愛知県,32,5,3.000000,2.000000,2.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
QnNgnN-IMFkKm-NgNQN9,2023年03月04日(土),東京都,48,10,3.000000,4.000000,4.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FkFff1-jjzi6Y-k5ff5f,2023年03月04日(土),茨城県,48,14,3.000000,3.000000,3.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
y2UppE-owyVQC-pSXXSp,2023年03月04日(土),茨城県,48,12,3.000000,2.000000,2.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fk1vFF-y6G0Mu-wffkf5,2023年03月04日(土),茨城県,48,15,3.000000,2.000000,2.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ccac8c-0nOJSk-c8c4DD,2023年03月04日(土),埼玉県,64,11,3.000000,3.000000,3.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [16]:
df_list[1]

,date,prefecture,num_players,rank,電氣發生器,森林封印石,高級球,巢穴球,離洞繩,講究腰帶,交替推車,學習裝置,天空封印石,能量回收器,寶可夢交替,健行鞋,寶可裝置3.0,飛羽球,放逐吸塵器,對戰VIP參加證,朋友手冊,洗翠的沉重球,等級球,消災手套,交錯切換機,活力頭帶,葉隱披風,山谷回音喇叭,野餐籃,不服輸頭帶,超級球,捕獲香氛,寶可夢補捉器,機器鶘,海綿手套,消除香水,一擊的捲軸 牙龍之卷,一擊的卷軸 貫通之卷,一擊的卷軸 憤怒之卷
pick_rate,NaN,NaN,<NA>,<NA>,1.000000,0.985866,0.978799,0.901060,0.756184,0.590106,0.452297,0.293286,0.208481,0.148410,0.120141,0.120141,0.113074,0.091873,0.084806,0.056537,0.04947,0.045936,0.042403,0.021201,0.017668,0.017668,0.014134,0.010601,0.007067,0.007067,0.007067,0.007067,0.007067,0.003534,0.003534,0.003534,0.003534,0.003534,0.003534
avg_num_used,NaN,NaN,<NA>,<NA>,3.989399,1.659498,3.613718,2.768627,1.574766,1.245509,1.554688,1.192771,1.000000,1.071429,1.588235,3.058824,2.125000,1.384615,1.041667,1.937500,1.00000,1.000000,2.083333,1.166667,4.000000,1.200000,1.250000,1.000000,1.000000,1.000000,2.000000,1.500000,2.500000,2.000000,1.000000,1.000000,2.000000,4.000000,4.000000
FFff5f-yNt4hB-kwF1Ff,2023年03月04日(土),愛知県,32,5,4.000000,2.000000,4.000000,3.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
QnNgnN-IMFkKm-NgNQN9,2023年03月04日(土),東京都,48,10,4.000000,2.000000,4.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FkFff1-jjzi6Y-k5ff5f,2023年03月04日(土),茨城県,48,14,4.000000,1.000000,3.000000,2.000000,0.000000,1.000000,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
y2UppE-owyVQC-pSXXSp,2023年03月04日(土),茨城県,48,12,4.000000,1.000000,4.000000,3.000000,2.000000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fk1vFF-y6G0Mu-wffkf5,2023年03月04日(土),茨城県,48,15,4.000000,1.000000,4.000000,3.000000,2.000000,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ccac8c-0nOJSk-c8c4DD,2023年03月04日(土),埼玉県,64,11,4.000000,2.000000,4.000000,2.000000,0.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FkkfVf-kjWtV0-wffFFV,2023年03月04日(土),埼玉県,80,3,4.000000,2.000000,4.000000,3.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ppE2XX-lNlmY3-pypXyX,2023年03月04日(土),埼玉県,80,4,4.000000,2.000000,4.000000,3.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
df_list[2]

,date,prefecture,num_players,rank,博士的研究,老大的指令,裁判,莎莉娜,派帕,奇巴納,阿克羅瑪的實驗,賽寶利,杜娟,克拉拉,米莫莎,希嘉娜的決心,芽米,捩木,皮歐尼,可爾妮的氣勢,珠貝,冒險家的發現,吉尼亞,工人,史帕克,牡丹
pick_rate,NaN,NaN,<NA>,<NA>,0.985866,0.982332,0.879859,0.816254,0.720848,0.303887,0.247350,0.116608,0.074205,0.042403,0.017668,0.017668,0.014134,0.014134,0.007067,0.007067,0.007067,0.003534,0.003534,0.003534,0.003534,0.003534
avg_num_used,NaN,NaN,<NA>,<NA>,3.752688,1.809353,2.112450,1.731602,1.725490,1.069767,1.942857,1.151515,1.047619,1.083333,1.000000,1.400000,1.000000,1.000000,2.000000,2.500000,1.000000,4.000000,2.000000,1.000000,1.000000,1.000000
FFff5f-yNt4hB-kwF1Ff,2023年03月04日(土),愛知県,32,5,4.000000,1.000000,1.000000,2.000000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
QnNgnN-IMFkKm-NgNQN9,2023年03月04日(土),東京都,48,10,1.000000,2.000000,2.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FkFff1-jjzi6Y-k5ff5f,2023年03月04日(土),茨城県,48,14,4.000000,1.000000,3.000000,2.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
y2UppE-owyVQC-pSXXSp,2023年03月04日(土),茨城県,48,12,4.000000,2.000000,1.000000,1.000000,2.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fk1vFF-y6G0Mu-wffkf5,2023年03月04日(土),茨城県,48,15,4.000000,1.000000,1.000000,2.000000,3.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ccac8c-0nOJSk-c8c4DD,2023年03月04日(土),埼玉県,64,11,4.000000,2.000000,1.000000,2.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FkkfVf-kjWtV0-wffFFV,2023年03月04日(土),埼玉県,80,3,4.000000,2.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ppE2XX-lNlmY3-pypXyX,2023年03月04日(土),埼玉県,80,4,4.000000,1.000000,2.000000,2.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [18]:
df_list[3]

,date,prefecture,num_players,rank,海灘場地,崩塌的競技場,神奧神殿,暴風雨山脈,放逐市,寶可補給站,通頂雪道
pick_rate,NaN,NaN,<NA>,<NA>,0.957597,0.173145,0.084806,0.074205,0.014134,0.010601,0.007067
avg_num_used,NaN,NaN,<NA>,<NA>,2.760148,1.000000,1.208333,1.666667,1.250000,3.000000,1.500000
FFff5f-yNt4hB-kwF1Ff,2023年03月04日(土),愛知県,32,5,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
QnNgnN-IMFkKm-NgNQN9,2023年03月04日(土),東京都,48,10,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FkFff1-jjzi6Y-k5ff5f,2023年03月04日(土),茨城県,48,14,2.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
y2UppE-owyVQC-pSXXSp,2023年03月04日(土),茨城県,48,12,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
fk1vFF-y6G0Mu-wffkf5,2023年03月04日(土),茨城県,48,15,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ccac8c-0nOJSk-c8c4DD,2023年03月04日(土),埼玉県,64,11,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FkkfVf-kjWtV0-wffFFV,2023年03月04日(土),埼玉県,80,3,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ppE2XX-lNlmY3-pypXyX,2023年03月04日(土),埼玉県,80,4,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [19]:
df_list[4]

,date,prefecture,num_players,rank,基本雷能量,雙重渦輪能量,基本火能量,基本鬥能量
pick_rate,NaN,NaN,<NA>,<NA>,1.000000,0.169611,0.003534,0.003534
avg_num_used,NaN,NaN,<NA>,<NA>,13.657244,1.000000,2.000000,1.000000
FFff5f-yNt4hB-kwF1Ff,2023年03月04日(土),愛知県,32,5,13.000000,1.000000,0.000000,0.000000
QnNgnN-IMFkKm-NgNQN9,2023年03月04日(土),東京都,48,10,14.000000,0.000000,0.000000,0.000000
FkFff1-jjzi6Y-k5ff5f,2023年03月04日(土),茨城県,48,14,14.000000,0.000000,0.000000,0.000000
y2UppE-owyVQC-pSXXSp,2023年03月04日(土),茨城県,48,12,13.000000,1.000000,0.000000,0.000000
fk1vFF-y6G0Mu-wffkf5,2023年03月04日(土),茨城県,48,15,13.000000,1.000000,0.000000,0.000000
ccac8c-0nOJSk-c8c4DD,2023年03月04日(土),埼玉県,64,11,14.000000,0.000000,0.000000,0.000000
FkkfVf-kjWtV0-wffFFV,2023年03月04日(土),埼玉県,80,3,13.000000,1.000000,0.000000,0.000000
ppE2XX-lNlmY3-pypXyX,2023年03月04日(土),埼玉県,80,4,13.000000,1.000000,0.000000,0.000000


In [20]:
len(decks["others"])

91